[Retour au sommaire](../index.ipynb)

# TP Stéganographie

![](img/hed-steganography.avif)

La stéganographie est l'art de la dissimulation.

Dans ce TP nous allons dissimuler une image dans une autre.

- L'image qui reçoit est appelée **l'image hôte**.
- L'image qui doit se dissumuler est appelée **l'image invitée**.

Nous pourrions cacher :

- du texte dans une image
- une image dans un son
- un son dans une image...

le principe resterait le même.

## Principe

Toutes les images matricielles contiennent des pixels.

Chaque pixel d'une image est composée de 3 composantes : Rouge, Vert et Bleu. (RVB)

Si on prend une image codée sur 24bits (3 octets), chaque composante de la couleur est donc codée sur 1 octet.

Il y a donc $2^8$ possibilités pour chaque composante:

- 256 nuances de rouge possibles (de 0 à 255)
- 256 nuances de vert possibles (de 0 à 255)
- 256 nuances de bleu possibles (de 0 à 255)

Dans ce type d'image sur 24 bits, il y a donc $2^{24} \approx 16 700 000$ couleurs possibles.


Considérons, **une composante** d'**un pixel** de l'image:

Si on modifie, les **deux bits de poids faible**, au maximum, on aura une différence de 3 avec la composante rouge d'origine : 

 - si j'ai 00 -> 11 (3 de difference maximale)
 - si j'ai 01 -> 11 (2 de différence maximale)
 - si j'ai 10 -> 00 (2 de difference maximale)
 - si j'ai 11 -> 00 (3 de difference maximale)
 

On obtient donc, **dans le pire des cas**, une différence de 3 par rapport à la valeur initiale. Et ce sur 256 nuances possibles.
 
 **La différence est si minime qu'il est impossible de la voir à l'oeil**
 
 Voici par exemple des couleurs **uniquement à base de vert**  (R=0, B=0):
 
 <table style="width:40%">
    <tr style="border:1px solid black">
        <td><strong>Un</strong> bit de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 129, 0);padding:25px">Code RGB(0, 129, 0)</td>
    </tr>
    <tr style="border:1px solid black">
        <td><strong>Deux</strong> bits de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 131, 0);padding:25px">Code RGB(0, 131, 0)</td>
    </tr>
    <tr style="border:1px solid black">
        <td><strong>Trois</strong> bits de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 135, 0);padding:25px">Code RGB(0, 135, 0)</td>
    </tr>
    <tr style="border:1px solid black">
        <td><strong>Quatre</strong> bits de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 143, 0);padding:25px">Code RGB(0, 143, 0)</td>
    </tr>
    <tr style="border:1px solid black">
        <td><strong>Cinq</strong> bits de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 159, 0);padding:25px">Code RGB(0, 159, 0)</td>
    </tr>
    <tr style="border:1px solid black">
        <td><strong>Six</strong> bits de poids faible de différence</td>
        <td style="background-color: rgb(0, 128, 0);padding:25px">Code RGB(0, 128, 0)</td>
        <td style="background-color: rgb(0, 191, 0);padding:25px">Code RGB(0, 191, 0)</td>
    </tr>
 </table>


 
<div class="alert alert-info">Nous allons nous contenter de modifier <b>les deux derniers bits</b> des composantes R, V et B de chaque pixel de notre image hôte afin exploiter cette 'réserve' de bits pour <b>cacher</b> notre seconde image.</div>

## Principes utilisés lors de ce TP

### Phase de dissimulation

On utilise une image **hôte** et une image **invitée** de **dimensions égales**.

Les bits de poids forts (MSB : **M**ost **S**ignificant **B**it) sont situés à gauche (128/0 pour le premier, 64/0 pour le second)

Les bits de poids faibles (LSB : **L**ess **S**ignificant **B**it) sont situés à droite (1/0 pour le premier, 2/0 pour le second)

<div class="alert alert-info">Ainsi, si je conserve uniquement les deux bits de poids fort de chacune des 3 composantes de chaque pixel de l'image invitée, je conserverai <b>grossièrement</b> les couleurs d'origine .

Si je "cache" ces deux bits de poids fort dans les deux bits de poids faible de mon image hôte, je ne changerai pratiquement pas la valeur de la composante de l'hôte.</div>


La stratégie est donc la suivante : Pour chaque pixel des images, et pour chaque composante de couleur :

- Je récupère les deux bits de poids fort de l'image invitée;
- Je remplace les deux bits de poids faible de l'hôte par les valeurs des 2 bits de poids fort de l'invité.

![schéma](img/stegano_schema1.png)

**Remarque**

On peut donc stocker uniquement la valeur des 3 composantes de l'image à cacher sur 6 bits soit: 


**$2^6=64$ couleurs possibles pour notre image dissimulée.**

Lors de la phase d'extraction, l'image qui se révèlera sera donc dégradée en terme de nombres de couleurs.

### Phase de d'extraction

- On crée une image de la même dimension
- On parcourt tous les pixels de l'image qui contient notre image dissimulée.
  - On recupère les deux dernièrs bits de chaque composante RVB
  - Puis on ajoute 6 bits derrière chaque composante. A tester faut il ajouter :
    - 000000
    - 111111
    - 101010 ?
  - on injecte ces valeurs dans le pixel de notre image blanche.
- on retourne notre image blanche (qui ne l'est plus...)



### Echauffement...

#### Comprendre comment extraire les valeurs de toutes les composantes (R, V, B) de tous les pixels de l'image

Voici une image de deux pixels sur trois pixels grossie :

<img src="img/image_3_2_grossie.png">

Nous allons utiliser la librairie [PIL](https://pillow.readthedocs.io/en/stable/) ( **P**ython **I**mage **L**ibrairie ) afin de parcourir les pixels de cette image.

In [ ]:
from PIL import Image # importation du module Image de la librairie PIL

image = Image.open("img/image_3_2.bmp") # je précise le chemin de mon image
largeur, hauteur = image.size # image.size retourne un tuple de 2 valeurs

print(f"largeur = {largeur} pixels")
print(f"hauteur = {hauteur} pixels")

#pixels = image.load() # j'ai tous les pixels de l'image grace à load()

#for y in range(hauteur): # je boucle sur la hauteur
#    print("------------------------------> NOUVELLE LIGNE")
#    for x in range(largeur): # je boucle sur la largeur
#        print(f"\nJe suis sur le pixel de coordonnées ({x},{y})")
        ### On récupère les valeurs r, v, b d'un pixel en donnant les coord (x, y) du pixel
        ### pixels est un peu comme un tableau de tableau
#        r, v, b = pixels[x, y] 
#        print(f"composante rouge = {r} = {bin(r)}")
#        print(f"composante verte = {v} = {bin(v)}")
#        print(f"composante bleue = {b} = {bin(b)}")


#### Comprendre comment fusionner deux composantes (l'hôte et l'invité)

Imaginons que 

- la composante rouge du pixel de coordonnées (x, y) de l'**hôte** est 224
- la composante rouge du pixel de coordonnées (x, y) de l'**invité** est 190

In [ ]:
r_hote = 224
r_invite = 190
### Je passe en binaire
r_hote_bin = bin(r_hote)
r_invite_bin = bin(r_invite)
print(f"La valeur de l'hôte en binaire est {r_hote_bin}")
print(f"La valeur de l'invité en binaire est {r_invite_bin}")

### bin(nombre) renvoie une string, j'enleve le '0b'
#r_hote_bin = r_hote_bin[2:]
#r_invite_bin = r_invite_bin[2:]
#print(f"La valeur de l'hôte est {r_hote_bin}.")
#print(f"La valeur de l'invité est {r_invite_bin}.")

### J'enleve les deux derniers bits de l'hote (LSB)
#r_hote_bin = r_hote_bin[:-2]
#print(f"La valeur tronquée de l'hôte est maintenant de {r_hote_bin}.")
### Je garde les deux premiers bits de l'invité (MSB)
#r_invite_bin = r_invite_bin[0:2]
#print(f"les deux premiers bits de l'invité sont {r_invite_bin}.")

#fusion = r_hote_bin + r_invite_bin
#print(f"La fusion donne {fusion}.")

### Ne reste plus qu'à repasser cela en nombre.

#valeur_fusion = int(fusion, 2) # le 2 est pour le binaire
#print(f"Le nombre fusionné est maintenant {valeur_fusion}.")


#### Tout se passait si bien...

Prenons maintenant : 

- r_hote = 224
- r_invite = 6




In [ ]:
r_hote = 224
r_invite = 6
### Je passe en binaire
r_hote_bin = bin(r_hote)
r_invite_bin = bin(r_invite)
print(f"La valeur de l'hôte est {r_hote_bin}")
print(f"La valeur de l'invité est {r_invite_bin}")

print("Aïe Aïe Aïe... : regardez la valeur de l'invité !!! Vous avez compris le problème ?")
print("+------------------------------------------------+")
print("|  J'espère car c'est à vous de le résoudre :-)  |")
print("+------------------------------------------------+")


### Déroulement du TP

#### Phases de préparation

- Créer un python virtuel *stegano* et configurer Thonny afin d'utiliser le python virtuel nouvellement créé. Voir [Pratique commune / python virtuel](../../../Pratique_commune/venv.ipynb).
- Allez sur https://github.com/saintlouis29/stegano et faire un fork de ce projet dans votre espace github;
- Depuis un terminal, aller dans le répertoire de vos TPs/Projets puis cloner votre fork (git clone adresse_de_votre_fork);
- Depuis le terminal, se rendre dans le dossier cloné;
- Installer votre projet dans votre python virtuel : "*pip install --editable .*"
  
#### Ecriture de la fonction *fusion_composantes*

- Réfléchir à la façon de fusionner la composante de l'invité dans la composante de l'hôte;
- Coder et s'aider des tests : si les tests passent, votre fonction est implémentée.

#### Ecriture de la fonction *dissimuler*

- Choisir deux images sur le web, avec gimp les retailler à la même définition (taille) et les convertir en bmp;
- S'aider des exemples de code plus haut dans ce TP;
- Evidemment, appeler la fonction *fusion_composantes* dans la fonction *dissimuler*.

#### Ecriture de la fonction *extraction_composante*

- Coder et s'aider des tests : si les tests passent, votre fonction est implémentée.

#### Ecriture de la fonction *reveler*

- A partir de l'image obtenue avec la fonction *dissimuler*, extraire l'image qui y est dissimulée.

#### Améliorations possibles

**Le nombre de bits de poids faible est codé en dur à 2, améliorons cela :**

- modifier les signatures des fonctions *fusion_composantes* et *dissimuler* afin que ceci soit un paramètre nommé *nlsb* dont la valeur par défaut est 2;
- intégrer cete variable *nlsb* dans le code de ces deux fonctions;
- faire de même pour les fonctions *extraction_composante* et *reveler*.

**Lors de la révélation de l'image il faut spécifier le nombre de bits de poids faible à utiliser, pourquoi ne pas dissiumuler également cette info ?**

- Dans la fonction *dissimuler*, ajouter le paramètre nommé **hide_nlsb_in_first_pixel** dont la valeur par défaut est **False**. Lors de l'appel de la fonction, si ce paramètre est à True, la valeur de **nlsb** sera dissimulée dans la composante rouge du pixel (0,0).
- Dans la fonction *reveler*, ajouter le paramètre nommé **extract_nlsb_in_first_pixel** dont la valeur par défaut est **False**. Lors de l'appel de la fonction, si ce paramètre est à True, la valeur de **nlsb** sera extraite de la composante rouge du pixel (0,0) et utilisée pour extraire l'image cachée.



## Un autre principe possible

Plutôt que de diminuer le nombre de couleurs, nous allons diminuer la définition de notre image invitée.

Si on divise par 2, la largeur et la hauteur, le nombre de pixels est divisé par 4.

Voici un schéma pour la composante rouge d'un pixel: 

![schéma](img/stegano_schema.png)

Les 8 bits de la composante Rouge sont dissimulés par paquet de 2 dans la composante rouge de 4 pixels de l'hôte.

**Notre image invitée conservera la qualité de ses couleurs mais sera 4 fois plus petite.**



[Retour au sommaire](../index.ipynb)